In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "36e1a0a6",
   "metadata": {},
   "source": [
    "# Exploratory Data Analysis"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "bfed1735",
   "metadata": {},
   "source": [
    "### Objective and Hypotheses:"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "15b7ad61",
   "metadata": {},
   "source": [
    "#### To analyze the price relations between certain cryptocurrencies and the equities market over a 32 months period and to determine possible correlations.\n",
    "\n",
    "#### Hypothesis (1): Price of cryptocurrencies (BTC and ETH) strongly correlate, but do not correlate with the broader equities market (SPY).\n",
    "\n",
    "#### Hypothesis (2): Price of blockchain related stocks correlates better with BTC than with the prices of other stocks."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "a3f981b2",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Don't modify this cell.\n",
    "# Dependencies and Setup\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "\n",
    "#import statistics\n",
    "import requests\n",
    "import time\n",
    "#import math\n",
    "#from scipy import stats\n",
    "from pprint import pprint\n",
    "\n",
    "# Import API key\n",
    "from api_keys import api_key\n",
    "\n",
    "#import ready packages\n",
    "from alpha_vantage.timeseries import TimeSeries\n",
    "from alpha_vantage.cryptocurrencies import CryptoCurrencies"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4358b9f4",
   "metadata": {},
   "source": [
    "#### The above code imports all packages we might need¶ "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "5b59b1eb",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>BTC (USD)</th>\n",
       "      <th>ETH (USD)</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>date</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>2021-11-11</th>\n",
       "      <td>64535.8</td>\n",
       "      <td>4592.74</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "            BTC (USD)  ETH (USD)\n",
       "date                            \n",
       "2021-11-11    64535.8    4592.74"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "#Using existing alpha_vantage packages:\n",
    "\n",
    "def get_crypto(sym, mar): # function takes the symbol of the crypto and the market (US) as parameters\n",
    "    ccb = CryptoCurrencies(key=api_key, output_format='pandas')\n",
    "    data, meta_data = ccb.get_digital_currency_daily(symbol=sym, market=mar)\n",
    "    data = data.drop(columns = ['1a. open (USD)', '1b. open (USD)', '2a. high (USD)', '2b. high (USD)',\n",
    "       '3a. low (USD)', '3b. low (USD)', '4a. close (USD)',\n",
    "       '5. volume', '6. market cap (USD)']) # we drop all columns we do not need, keep only the close price\n",
    "    data = data.rename(columns = {'4b. close (USD)' : f'{sym} (USD)'}) # rename column appropriately\n",
    "    return data, meta_data\n",
    "\n",
    "b_data, b_meta_data = get_crypto('BTC', 'USD') # request and load data using our function\n",
    "e_data, e_meta_data = get_crypto('ETH', 'USD')\n",
    "\n",
    "crypto_data = pd.concat([b_data, e_data], axis = 1) # put both into the same dataframe for ease of use\n",
    "crypto_data.head(1)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "2ed4b120",
   "metadata": {},
   "source": [
    "#### Here we defined a function that requests and loads the crypto data we will need"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "1b66d786",
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_correlations(df, roll, location1, location2, delta, fsize, lsize):\n",
    "    columns = df.columns.to_list()\n",
    "# this function takes multiple parameters, the dataframe to work with, the rolling period for the rolling correlation,\n",
    "# the (x coordinate) location of the line representing the global correlation, the (x coordinate) location of the anno-\n",
    "# tation, their difference (delta) in y coordinates and title fontsize as well as tick labelsize.\n",
    "    \n",
    "    fig = plt.figure(figsize = (22, 18)) # create figure\n",
    "    \n",
    "    ax0 = fig.add_subplot(3, 1, 1) # create 3 subplots in a column\n",
    "    ax2 = fig.add_subplot(3, 1, 2)\n",
    "    ax3 = fig.add_subplot(3, 1, 3, sharex = ax2) #\n",
    "    \n",
    "    fig.tight_layout(pad = 7.0) # set the spacing between plots\n",
    "    \n",
    "    ax0.scatter(df[columns[0]], df[columns[1]], c = df.index, cmap = 'viridis') # create scatterplot using the 2 columns of df, set color hue by index (which is the date)\n",
    "    ax0.set_xlabel(f'Price of {columns[0]}', color = 'r', fontsize = fsize) # set labels with appropriate color and size\n",
    "    ax0.set_ylabel(f'Price of {columns[1]}', color = 'b', fontsize = fsize)\n",
    "    ax0.set_title(f'Scatterplot of {columns[0]} and {columns[1]} Prices, Hues Representing Time, Lighter more Recent', fontsize = fsize)\n",
    "    ax0.tick_params('x', colors = 'r', labelbottom = True, labelsize = lsize) # set the tick parameters\n",
    "    ax0.tick_params('y', colors = 'b', labelsize = lsize)\n",
    "    \n",
    "    ax2.plot(df.index, df[columns[0]], c = 'r') # plot time series of one asset\n",
    "    ax2.set_xlabel('Date', fontsize = fsize)\n",
    "    ax2.set_ylabel(f'Price of {columns[0]}', color = 'r', fontsize = fsize)\n",
    "    ax2.set_title(f'Time Series of {columns[0]} and {columns[1]} Prices', fontsize = fsize)\n",
    "    ax2.tick_params('y', colors = 'r', labelsize = lsize)\n",
    "    ax2.tick_params('x', labelbottom = True, labelsize = lsize)\n",
    "\n",
    "    ax4 = ax2.twinx() # share axes with ax2 and plot on the same space (y axes labels will be different)\n",
    "    ax4.plot(df.index, df[columns[1]], c = 'b') # plot time series of other asset\n",
    "    ax4.set_ylabel(f'Price of {columns[1]}', color = 'b', fontsize = fsize)\n",
    "    ax4.tick_params('y', colors = 'b', labelbottom = True, labelsize = lsize)\n",
    "    \n",
    "    ax3.plot(df[columns[0]].rolling(window = roll).corr(df[columns[1]]).shift(periods = -(roll-1))) # plot rolling correlation coefficient, presented at the end of each (roll) day period\n",
    "    ax3.axhline(df.corr().iloc[0,1], c = 'r') # draw a red line showing the global correlation coefficient\n",
    "    ax3.tick_params('y', labelsize = lsize)\n",
    "    ax3.tick_params('x', labelsize = lsize)\n",
    "    ax3.set_xlabel('Date', fontsize = fsize)\n",
    "    ax3.set_ylabel('45 day rolling correlation between' '\\n' f'{columns[0]} and {columns[1]} prices', fontsize = fsize)\n",
    "    ax3.set_title(f'Rolling Correlation Analysis of {columns[0]} and {columns[1]} Prices', fontsize = fsize)\n",
    "    ax3.annotate(f'global correlation coefficient: {round(df.corr().iloc[0,1], 3)}', \n",
    "               xy = [pd.Timestamp(location1), df.corr().iloc[0,1]], xytext = [pd.Timestamp(location2), df.corr().iloc[0,1] - delta], arrowprops = {}, fontsize = 20) # set location and arrowtype of annotation\n",
    "    \n",
    "    fig.savefig(f'Analysis of {columns[0]} and {columns[1]} Prices.png') # save figure\n",
    "    print(df.corr())\n",
    "    "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9b42c18a",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}